# Imports

In [26]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import timedelta

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Read excel

In [27]:
df = pd.read_excel('online_retail_II.xlsx')
print(df.head())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


# Pre-Processing

In [28]:
# 2) Tipi e pulizia base
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], dayfirst=True, errors="coerce")
counter = df.isna().sum()
print(counter)
df = df.dropna(subset=["Customer ID"])

# 3) Rimozione cancellazioni/resi e valori non validi
# (nel dataset UCI i resi spesso hanno InvoiceNo che inizia per 'C' e quantità negative)
df = df[~df["Invoice"].astype(str).str.startswith("C")]
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]

# 4) Feature di importo riga
df["TotalPrice"] = df["Quantity"] * df["Price"]

# 5) CustomerID come stringa (evita .0 tipico del float)
df["Customer ID"] = df["Customer ID"].astype("int64").astype(str)

# 6) CLV = somma del totale speso per cliente
clv = (df.groupby("Customer ID", as_index=False)["TotalPrice"]
            .sum()
            .rename(columns={"TotalPrice": "CustomerLifetimeValue"})
            .sort_values("CustomerLifetimeValue", ascending=False))

# Stampa top 10
print("Top 10 clienti per CLV:")
print(clv.head(10).to_string(index=False))

# Salvataggio su disco
output_path = "customer_lifetime_value.csv"
clv.to_csv(output_path, index=False)
print(f"\nCLV salvato in: {output_path}")

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64
Top 10 clienti per CLV:
Customer ID  CustomerLifetimeValue
      18102              349164.35
      14646              248396.50
      14156              196566.74
      14911              152147.57
      13694              131443.19
      17511               84541.17
      15061               83284.38
      16684               80489.21
      16754               65500.07
      17949               60117.60

CLV salvato in: customer_lifetime_value.csv


# Feature Engineering

In [33]:
# === Blocco 2 – Feature engineering (aggiornato) ===
# (assume che 'df' sia il risultato del pre-processing)

customer_metrics = df.groupby("Customer ID").agg(
    CustomerLifetimeValue=("TotalPrice", "sum"),   # Totale speso (CLV)
    Frequency=("Invoice", "nunique"),            # Numero ordini distinti
    TotalItems=("Quantity", "sum"),                # Numero totale di articoli acquistati
    Country=("Country", lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])  # Paese più frequente
).reset_index()

# Spesa media per ordine
customer_metrics["AverageOrderValue"] = (
    customer_metrics["CustomerLifetimeValue"] / customer_metrics["Frequency"]
)

# Spesa media per articolo (opzionale, utile per insight)
customer_metrics["AveragePricePerItem"] = (
    customer_metrics["CustomerLifetimeValue"] / customer_metrics["TotalItems"]
)

# Ordina per CLV
customer_metrics = customer_metrics.sort_values("CustomerLifetimeValue", ascending=False)

# Visualizza top 10
customer_metrics.head(10)


,Customer ID,CustomerLifetimeValue,Frequency,TotalItems,Country,AverageOrderValue,AveragePricePerItem
4183,18102,349164.35,89,124216,United Kingdom,3923.194944,2.810945
1637,14646,248396.50,78,170278,Netherlands,3184.570513,1.458770
1269,14156,196566.74,102,108107,EIRE,1927.124902,1.818261
1840,14911,152147.57,205,69722,EIRE,742.183268,2.182203
939,13694,131443.19,94,125893,United Kingdom,1398.331809,1.044087
3744,17511,84541.17,31,55107,United Kingdom,2727.134516,1.534128
1951,15061,83284.38,86,51791,United Kingdom,968.423023,1.608086
3128,16684,80489.21,27,54555,United Kingdom,2981.081852,1.475377
3177,16754,65500.07,29,63551,United Kingdom,2258.623103,1.030669
4065,17949,60117.60,74,30112,United Kingdom,812.400000,1.996467


## Normalizzazione

In [34]:
features = ["CustomerLifetimeValue", "Frequency", "TotalItems", "AverageOrderValue", "AveragePricePerItem"]

scaler = StandardScaler()
customer_metrics_standardized = customer_metrics.copy()
customer_metrics_standardized[features] = scaler.fit_transform(customer_metrics[features])

customer_metrics_standardized.head(10)


,Customer ID,CustomerLifetimeValue,Frequency,TotalItems,Country,AverageOrderValue,AveragePricePerItem
4183,18102,38.942969,10.349069,19.034327,United Kingdom,7.197609,-0.027890
1637,14646,27.637820,9.002559,26.166417,Netherlands,5.697885,-0.034541
1269,14156,21.823038,11.940399,16.540062,EIRE,3.144732,-0.032773
1840,14911,16.839649,24.548631,10.596654,EIRE,0.738793,-0.030982
939,13694,14.516824,10.961119,19.293988,United Kingdom,2.071055,-0.036581
3744,17511,9.254885,3.249288,8.333715,United Kingdom,4.769094,-0.034170
1951,15061,9.113886,9.981839,7.820276,United Kingdom,1.198156,-0.033806
3128,16684,8.800295,2.759648,8.248245,United Kingdom,5.284716,-0.034459
3177,16754,7.118663,3.004468,9.641156,United Kingdom,3.817815,-0.036647
4065,17949,6.514804,8.512918,4.463571,United Kingdom,0.881363,-0.031896
